In [17]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
!pip install segmentation-models-pytorch

!pip install onnx
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import segmentation_models_pytorch as smp

In [20]:
sample_num = 13
EPOCHS=50
BATCH_SIZE=32
LR=0.001

ratio=0.5

ENCODER='resnet50'
WEIGHTS='imagenet'

In [21]:
#defining the model
class SegmentationModel(nn.Module):
    def __init__(self):
        super(SegmentationModel,self).__init__()

        self.arc=smp.Unet(
            encoder_name=ENCODER,
            encoder_weights=WEIGHTS,
            in_channels=3,
            classes=1,
            activation=None
        )
    def forward(self, images, masks=None):
        logits = self.arc(images)
        #if masks != None:
            #lossDice = DiceLoss(mode='binary')(logits,masks)
            #return logits, lossDice
        return logits

In [22]:
model=SegmentationModel()
model.to(device)
model.load_state_dict(torch.load('best_model_unet_resnet50.pt'))

dummy_input = torch.randn(1, 3, 256, 256).to(device)

model.eval()
torch_out = model(dummy_input)

In [23]:
onnx_path = 'model.onnx'

torch.onnx.export(model,
                dummy_input,
                onnx_path,
                verbose=True,
                input_names = ['input'],   # the model's input names
                output_names = ['output'], # the model's output names
                dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                              'output' : {0 : 'batch_size'}})

/usr/local/lib/python3.10/dist-packages/segmentation_models_pytorch/base/model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:


============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [24]:
import onnx

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

In [25]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession(onnx_path)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_outs = ort_session.run(None, ort_inputs)

np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-02, atol=1e-04)